In [34]:
import numpy as np
import pandas as pd
import duckdb

In [35]:
from evaluation import *

In [36]:
from cf_model import sgd_mf, sgd_mmf, sgd_wmf, sgd_mwmf

# Import data

In [37]:
df_txn = pd.read_csv('./database/transactions.csv')
df_price = pd.read_csv('./database/close_prices.csv')

In [38]:
df_txn = duckdb.query("""
select yyyymm, customerID, ISIN, transactionType, sum(units) as unit
from 
    (
    select *
    , year(cast(timestamp as date))*100 + month(cast(timestamp as date)) as yyyymm
    from df_txn
    where transactionType = 'Buy'
    ) a
group by yyyymm, customerID, ISIN, transactionType
""").to_df()

In [39]:
df_return = duckdb.query("""
select isin, yyyymm
, (closePrice-closePrice_lag)/closePrice_lag as daily_return
from
    (
    select *
    , year(cast(timestamp as date))*100 + month(cast(timestamp as date)) as yyyymm
    , lag(closePrice,1) over(partition by isin order by cast(timestamp as date)) closePrice_lag
    from df_price
    ) a
where closePrice_lag is not null
""").to_df()

# Create template

In [40]:
unique_customerid = df_txn['customerID'].unique()[:100]
unique_isin = df_txn['ISIN'].unique()[:]

In [41]:
df_customer = pd.DataFrame(unique_customerid, columns=['customerID'])
df_isin = pd.DataFrame(unique_isin, columns=['ISIN'])

In [42]:
df_user_item_template = df_customer.merge(df_isin, how='cross')

# Split train-val-test (user-item)

In [43]:
yyyy_list = [201912, 202006, 202012, 202106, 202112, 202206]

In [44]:
fold = 5

In [45]:
for i in range(fold):
    print('===================================')
    print('fold:', i+1)
    print('train:', 201801, yyyy_list[i])
    print('val:', 201801, yyyy_list[i+1])
    print('===================================')

    df_train = df_txn[(df_txn['yyyymm'] >= 201801) & (df_txn['yyyymm'] <= yyyy_list[i])]
    df_val = df_txn[(df_txn['yyyymm'] >= 201801) & (df_txn['yyyymm'] <= yyyy_list[i+1])]

    df_train = df_train.groupby(["customerID", "ISIN"])["ISIN"].nunique().reset_index(name="action")
    df_val = df_val.groupby(["customerID", "ISIN"])["ISIN"].nunique().reset_index(name="action")

    df_train = pd.merge(df_user_item_template, df_train, on=['customerID','ISIN'], how='left').fillna(0)
    df_val = pd.merge(df_user_item_template, df_val, on=['customerID','ISIN'], how='left').fillna(0)

    train_array = df_train.pivot_table(index='customerID', columns='ISIN', values='action', fill_value=0).values
    val_array = df_val.pivot_table(index='customerID', columns='ISIN', values='action', fill_value=0).values

    break

fold: 1
train: 201801 201912
val: 201801 202006


In [46]:
train_array.shape, val_array.shape

((100, 320), (100, 320))

# Split train-val-test (mean)

In [47]:
yyyy_list = [201906, 201912, 202006, 202012, 202106, 202112, 202206]

In [48]:
fold = 5

In [49]:
for i in range(fold):
    print('===================================')
    print('fold:', i+1)
    print('train:', yyyy_list[i], yyyy_list[i+1])
    print('val:', yyyy_list[i+1], yyyy_list[i+2])
    print('===================================')

    df_train_return = df_return[(df_return['yyyymm'] > yyyy_list[i]) & (df_return['yyyymm'] <= yyyy_list[i+1])]
    df_val_return = df_return[(df_return['yyyymm'] > yyyy_list[i+1]) & (df_return['yyyymm'] <= yyyy_list[i+2])]
    
    train_mean_df = df_train_return.groupby("ISIN")["daily_return"].mean().reset_index()
    val_mean_df = df_val_return.groupby("ISIN")["daily_return"].mean().reset_index()

    train_mean_df = pd.merge(df_isin, train_mean_df, on=['ISIN'], how='left').fillna(0)
    val_mean_df = pd.merge(df_isin, val_mean_df, on=['ISIN'], how='left').fillna(0)

    train_mean_array = train_mean_df['daily_return'].values
    val_mean_array = val_mean_df['daily_return'].values

    break

fold: 1
train: 201906 201912
val: 201912 202006


# Train model

In [50]:
pred_array_mf = sgd_mf(train_array, K=50, epochs=1000)

In [51]:
pred_array_mmf = sgd_mmf(train_array, train_mean_array, K=50, epochs=1000)

In [52]:
pred_array_wmf = sgd_wmf(train_array, K=50, epochs=1000)

In [53]:
pred_array_mwmf = sgd_mwmf(train_array, train_mean_array, K=50, epochs=1000)

# Evaluation

In [54]:
pred_top_k_mf = get_top_k(pred_array_mf, 5)
mapk(val_array, pred_top_k_mf, k=5)

0.0034000000000000002

In [55]:
pred_top_k_mmf = get_top_k(pred_array_mmf, 5)
mapk(val_array, pred_top_k_mmf, k=5)

0.0006666666666666666

In [56]:
pred_top_k_wmf = get_top_k(pred_array_wmf, 5)
mapk(val_array, pred_top_k_wmf, k=5)

0.8055

In [57]:
pred_top_k_mwmf = get_top_k(pred_array_mwmf, 5)
mapk(val_array, pred_top_k_mwmf, k=5)

0.8132